In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import subprocess
import shlex
import numpy as np
import pandas as pd

import os
import sys

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib

repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.append(repo_root)

from src.utils.file_handlers import group_dataset_files, read_dataset_files
from src.utils.operation_helpers import run_models, split_games
from src.utils.c_operation_helpers import  run_simulation

Python

In [6]:
data_dir = os.path.join(repo_root, 'datasets', 'Real_Data')
grouped = group_dataset_files(data_dir)

data, pi_values = read_dataset_files(grouped['00103'], data_dir, is_synthetic=False)

# train, test = train_test_split(data, train_size=.8)
train, test = split_games(data, .8)

df = run_models(train, test, pi_values)
print(df)


         model  log-likelihoods  leadership-log-likelihood
0          BIN        -1.114381                  -0.797482
1         BINL        -1.157404                  -0.847886
2        HO_BT        -1.125124                  -0.796702
3       HOL_BT        -1.165548                  -0.844139
4  Spring_Rank        -1.106598                  -0.808997
5    Page_Rank        -1.320841                  -0.897634
6   Point_Wise        -1.385558                  -1.065916


C 

In [4]:
data_dir = os.path.join(repo_root, 'datasets', 'Real_Data')
grouped = group_dataset_files(data_dir)

edge_file = grouped['00103']['edges']
node_file = grouped['00103']['nodes']

edge_path = os.path.join(data_dir, edge_file)
node_path = os.path.join(data_dir, node_file)

results = run_simulation(node_path, edge_path, .8)
print(results)

  model  av_error    spearman     kendall     prior   HO_Like   HOL_Like  \
0    HO  0.354359   -0.142886   -0.108434   -1.5472  -2.74475  -0.832101   
1   HOL  0.323742  -0.0421654  -0.0302674  -1.48279   -2.7423  -0.823259   
2   BIN    0.3669   -0.140197   -0.108434  -1.58531  -2.79209  -0.843856   
3  BINL  0.342522  -0.0493774  -0.0408463  -1.54139  -2.78931  -0.837696   

  iterations  
0         10  
1          9  
2         10  
3         10  


In [ ]:


def process_results(base_path):
    # Read the summary CSV files
    log_likelihood_df = pd.read_csv(os.path.join(base_path, 'log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()
    leadership_log_likelihood_df = pd.read_csv(os.path.join(base_path, 'leadership_log_likelihood_summary.csv')).groupby(by=['dataset']).mean().reset_index()

    filtered_log_like = log_likelihood_df.drop(columns=['rep'])
    filtered_leader_like = leadership_log_likelihood_df.drop(columns=['rep'])
    
    log_like = subtract_columns(filtered_log_like, 3)
    leadership_log = subtract_columns(filtered_leader_like, 4)

    return log_like, leadership_log

def subtract_columns(df, compared_col):
    columns = df.columns.tolist()
    base_column = columns[compared_col] 
    print(base_column)
    for col in columns[1:]:
        
        df[col] = df[col] - df[base_column] 
    
    return df


def visualize_df(df, title):
    print(f"\n{title}\n")
    styled_df = df.style.set_table_styles(
    ).set_properties(**{
        'background-color': 'LightGray',
        'color': 'black',
        'border-color': 'black',
        'border-style': 'solid',
        'border-width': '1px',
        'text-align': 'left'
    })
    

    # print(df.to_latex(escape=True))
    display(styled_df)
   

In [ ]:
unique_datasets = pd.read_csv(os.path.join(os.path.join(os.getcwd(), 'results'), 'log_likelihood_summary.csv'))['dataset'].unique()
dataset_info = pd.read_csv(os.path.join(repo_root, 'datasets', 'dataset_info.csv'))
filtered_dataset_info = dataset_info[dataset_info['dataset_id'].isin(unique_datasets)].set_index('dataset_id')

domain_order = ["Election", "Sport", "Preferences", "Other"]
filtered_dataset_info['Domain'] = pd.Categorical(filtered_dataset_info['Domain'], categories=domain_order, ordered=True)

filtered_dataset_info = filtered_dataset_info.sort_values(by=['Domain', 'K1', 'K2', 'Name'])

visualize_df(filtered_dataset_info, 'Datasets Tested')

In [ ]:
log_like, leadership_log = process_results(os.path.join(os.getcwd(), 'results'))

def rename_df(df):
    merged_df = df.merge(dataset_info, left_on='dataset', right_on='dataset_id').drop(columns=['dataset_id','dataset'])
    # merged_df = merged_df.rename(columns={'Name': 'dataset_name'}).set_index('dataset_name')

    domain_order = ["Election", "Sport", "Preferences", "Other"]
    merged_df['Domain'] = pd.Categorical(merged_df['Domain'], categories=domain_order, ordered=True)
    merged_df = merged_df.sort_values(by=['Domain', 'K1', 'K2', 'Name'])
    merged_df = merged_df[['Domain', 'Name', 'BT','BT_leadership', 'HO_BT', 'HOL_BT', 'Spring_Rank', 'Spring_Rank_Leadership', 'Page_Rank','Page_Rank_Leadership', 'Point_Wise']]

    return merged_df

In [ ]:

log_like = rename_df(log_like)
visualize_df(log_like, 'log likelihood')

In [ ]:
leadership_log = rename_df(leadership_log)
visualize_df(leadership_log, 'Leadership log likelihood')